Setup environment

In [ ]:
# Ensure src folder is importable
import sys
from pathlib import Path
import os

project_root = Path.cwd().parent  # adjust if notebook is in /notebooks
sys.path.append(str(project_root))

# Auto-reload changes in .py files
%load_ext autoreload
%autoreload 2


Imports

In [ ]:
import torch
import numpy as np
import pandas as pd
from datetime import datetime
import joblib
import json

from src.data.data_loader import CryptoDataLoader
from src.data.feature_engineering import FeatureEngineer
from src.models.train import TimeSeriesTrainer
from src.evaluation.metrics import evaluate_model


Parameters

In [ ]:
# User-defined parameters
TICKER = "BTC-USD"
MODEL_TYPE = "lstm"  # 'lstm' or 'transformer'
EPOCHS = 50
SEQUENCE_LENGTH = 60
TRAIN_MODEL = True
EVALUATE_MODEL = True
DEPLOY_MODEL = False
SAVE_DIR = "models"

# Create save directory if it doesn't exist
os.makedirs(SAVE_DIR, exist_ok=True)


Initialize Trainer

In [ ]:
trainer = TimeSeriesTrainer(model_type=MODEL_TYPE)

Prepare Data

In [ ]:
print(f"Preparing data for {TICKER}...")
train_loader, val_loader, test_loader, feature_scaler, target_scaler, feature_cols = trainer.prepare_data(
    ticker=TICKER,
    sequence_length=SEQUENCE_LENGTH
)
print(f"Number of features: {len(feature_cols)}")


Build Model

In [ ]:
input_size = len(feature_cols)

if MODEL_TYPE == 'lstm':
    model = trainer.build_model(
        input_size=input_size,
        hidden_size=64,
        num_layers=2,
        output_size=1,
        dropout=0.2,
        bidirectional=False,
        use_gru=False
    )
else:  # transformer
    model = trainer.build_model(
        input_size=input_size,
        d_model=64,
        nhead=4,
        num_encoder_layers=2,
        dim_feedforward=128,
        output_size=1,
        dropout=0.1
    )

print(f"{MODEL_TYPE.upper()} model built.")


Train Model

In [ ]:
if TRAIN_MODEL:
    print(f"Training {MODEL_TYPE} model...")
    
    save_path = os.path.join(SAVE_DIR, f"{TICKER}_{MODEL_TYPE}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pth")
    
    history = trainer.train(
        train_loader=train_loader,
        val_loader=val_loader,
        epochs=EPOCHS,
        lr=0.001,
        patience=10,
        save_path=save_path
    )
    
    trainer.plot_history()
    print(f"Model saved to {save_path}")


Evaluate Model

In [ ]:
if EVALUATE_MODEL:
    print(f"Evaluating {MODEL_TYPE} model...")
    metrics, predictions, actuals = trainer.evaluate(test_loader, target_scaler)
    
    print("Evaluation Metrics:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")
    
    trainer.plot_predictions(actuals, predictions, n=100)


Save Model & Scalers (for deployment)

In [ ]:
# Save model info
model_info = {
    'model_type': MODEL_TYPE,
    'input_size': input_size,
    'sequence_length': SEQUENCE_LENGTH,
    'feature_cols': feature_cols
}
with open(os.path.join(SAVE_DIR, 'model_info.json'), 'w') as f:
    json.dump(model_info, f)

# Save scalers
joblib.dump(feature_scaler, os.path.join(SAVE_DIR, 'feature_scaler.pkl'))
joblib.dump(target_scaler, os.path.join(SAVE_DIR, 'target_scaler.pkl'))

print("Model info and scalers saved.")


Optional Deployment

In [ ]:
if DEPLOY_MODEL:
    print("Starting API server...")
    from src.deployment.api import app
    import uvicorn
    
    uvicorn.run(app, host="0.0.0.0", port=8000)
